In [ ]:
#Note: This code is used to plot Figure 5(b-d) of the manscuript.

import matplotlib.pyplot as plt
#fig, ax = plt.subplots()
import time
import numpy as np
import torch
from torch import nn, optim, autograd
from math import pi
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
import torch.nn.functional as F


torch.manual_seed(123456)
np.random.seed(123456)

def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0.0)


class Unit3(nn.Module):
    def __init__(self, in_N, out_N,actf):
        super(Unit3, self).__init__()
        self.in_N = in_N
        self.out_N = out_N
        self.actf = actf
        self.L = nn.Linear(in_N, out_N)

    def forward(self, x):
        actf=self.actf
        x1 = self.L(x)
        if actf==0:
            x2 = torch.tanh(x1)
        elif actf==1:
            x2 = torch.sigmoid(x1) 
        elif actf==2:
            x2 = torch.relu(x1)
        elif actf==3:
            x2 = torch.selu(x1)
        elif actf==4:
            x2 = F.softmax(x1, dim=1)
        return x2
    
class NN3(nn.Module):
    def __init__(self, in_N, width1, depth1,width2, depth2,out_N,bn,dp,dprate,actf):
        super(NN3, self).__init__()
        self.width1 = width1
        self.width2 = width2
        self.depth1 = depth1
        self.depth2 = depth2
        self.bn = bn
        self.dp = dp
        self.dprate = dprate
        self.actf = actf
        self.in_N = in_N
        self.out_N = out_N
        self.stack = nn.ModuleList()
        self.stack.append(Unit3(in_N, width1[0],actf))
        if bn==1:
            self.stack.append(nn.BatchNorm1d(width1[0]))
        for i in range(1,depth1):
            self.stack.append(Unit3(width1[i-1], width1[i],actf))
        
        if dp==1:
            self.stack.append(nn.Dropout(p=dprate))
        if depth2==1:
            self.stack.append(Unit3(width1[i], width2[0],1)) 
        else:
            self.stack.append(Unit3(width1[i], width2[0],actf))    
            for i in range(1,depth2-1):
                self.stack.append(Unit3(width2[i-1], width2[i],actf))
            self.stack.append(Unit3(width2[depth2-2], width2[depth2-1],4)) 
            
    def forward(self, x):
        for i in range(len(self.stack)):
            x = self.stack[i](x)
        return x

activation=0
dropout=1
dropout_rate=0.29791
normalization=1
batch_size=1000
layers1=10
layers2=1
neurons=86
learning_rate=0.00065
L1=[neurons]*layers1
L2=[neurons]*layers2+[8]
model_h = NN3(35,L1,layers1,L2,layers2+1, 8,normalization,dropout,dropout_rate,0)


        
load=1
PATH="checkpoint/model-1406.pt"
if load==1:
    checkpoint = torch.load(PATH)
    model_h.load_state_dict(checkpoint['model_h_state_dict'])
    optimizer2 = optim.AdamW([{'params': model_h.parameters()}], lr=learning_rate) 
    optimizer2.load_state_dict(checkpoint['optimizer2_state_dict'])

import numpy as np
norm=np.load('norm.npy')[0]
norm[10,1]=1
norm[11,1]=1
norm[12,1]=1
norm[13,1]=1
norm[14,1]=1
norm[15,1]=1

Temp=1200 #temperature
A = np.loadtxt(f'Labels_7el_d=0.03_{Temp}K_extra.txt')[:, 10:]
comp=np.loadtxt(f'Labels_7el_d=0.03_{Temp}K_extra.txt')[:,1:8]
a=Temp*np.ones([A.shape[0],1])
A=np.append(A,a,axis=1)

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        A[i,j]=(A[i,j]-norm[j][0])/(norm[j][1]-norm[j][0])
        
n=[7,9,38,40,45,48,49,39,25,26,29,47,34,35,41,46]
A1=np.delete(A,n,axis=1)
model_h.eval()
xlo_test=A1
y = model_h(torch.from_numpy(xlo_test).float())
n=np.zeros([0,7])
for i in range(y.shape[0]):
    a=y[i,0]+y[i,2]+y[i,3]+y[i,4]+y[i,5]+y[i,6]+y[i,7]
    if a<0.10: #threshold of phase
        n=np.append(n,comp[[i],:],axis=0)
import matplotlib.pyplot as plt
import numpy as np

num_labels = 7
num_points = n.shape[0]
print('num_points=',num_points)